In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [45]:
# Read in and pre-process data
columns = ['txId', 'timestep']

class_names = ['Illicit', 'Legal']

# Name the columns without known names
for x in range(165) :
    columns.append('col'+ str(x))

# These are the output labels
classes = pd.read_csv('/kaggle/input/elliptic-data-set/elliptic_bitcoin_dataset/elliptic_txs_classes.csv')

# These are 167 columns of feature data
features = pd.read_csv('/kaggle/input/elliptic-data-set/elliptic_bitcoin_dataset/elliptic_txs_features.csv', names=columns)

# Flatten the data, append class to features
data = features.assign(result=classes['class'])

# Trim the data to include only labeled data. 
dataset = data[data['result'] != "unknown"]
dataset['result'] = pd.to_numeric(dataset.result) - 1

In [51]:
# 0 means illicit and 1 means licit
dataset['result'].value_counts() 

In [46]:
dataset.head()

In [8]:
# defining train and test datasets
data = dataset.sample(frac=0.8, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
# installing required dependencies
!pip install streamlit pycaret --ignore-installed llvmlite numpy==1.20

In [16]:
# to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# we only need pandas and pycaret
import pandas as pd
from pycaret.classification import *

In [17]:
# initializing the env
# categorical_features=cat_cols,
exp_1 = setup(data = data,
              session_id=123, 
              target = 'result', 
              normalize=True, 
              normalize_method='minmax',
              transformation=True,
              use_gpu=True,
              log_experiment=True,
              experiment_name='elliptic1',
              fix_imbalance = True,
              silent=True)

In [19]:
# Let's pick the model
# model code, cv fold, and rounding upto _ places.
rf = create_model('rf', fold = 5, round=4)

In [20]:
print(rf)

In [23]:
plot_model(rf, plot = 'confusion_matrix')

In [25]:
save_model(rf,'RF_Model_V1')

In [64]:
# 0 means illicit and 1 means licit
Label, Score = predict_model(rf, round=2)[['Label', 'Score']].iloc[0]

print("label: ",Label)
print("score: ",Score)